In [1]:
pip install pytube


Note: you may need to restart the kernel to use updated packages.


In [5]:
from pytube import YouTube

In [6]:
yt= YouTube("https://www.youtube.com/watch?v=286SRNds5hs")

LLM - langchain- embedding

임베딩

In [7]:
from langchain_openai import OpenAIEmbeddings


embeddings = OpenAIEmbeddings(model="text-embedding-3-small") #

text = "임베딩 테스트를 하기 위한 샘플 문장입니다."

In [10]:
query_result = embeddings.embed_query(text)


doc_result = embeddings.embed_documents([text, text, text, text])





embeddings_1024 = OpenAIEmbeddings(model="text-embedding-3-small", dimensions=1024)
embeddings_original = OpenAIEmbeddings(model="text-embedding-3-small")

len_1024 = embeddings_1024.embed_documents([text])
len_original = embeddings_original.embed_documents([text])


print(len(len_1024[0]))
print(len(len_original[0]))


1024
1536


In [11]:
from sklearn.metrics.pairwise import cosine_similarity

In [13]:
from sklearn.metrics.pairwise import cosine_similarity

sentence1 = "안녕하세요? 반갑습니다."
sentence2 = "안녕하세요? 반갑습니다!"
sentence3 = "안녕하세요? 만나서 반가워요."
sentence4 = "Hi, nice to meet you."
sentence5 = "I like to eat apples."

def similarity(a, b):
    return cosine_similarity([a], [b]) #

캐시 기능을 포함한 임베딩

In [14]:
from langchain.storage import LocalFileStore
from langchain_openai import OpenAIEmbeddings
from langchain.embeddings import CacheBackedEmbeddings
from langchain_community.vectorstores.faiss import FAISS
from langchain_community.document_loaders import TextLoader
from langchain_text_splitters import CharacterTextSplitter


embedding = OpenAIEmbeddings()
store = LocalFileStore("./cache/")

cached_embedder = CacheBackedEmbeddings.from_bytes_store(
    underlying_embeddings=embedding, # 기본 임베딩 모델 지정
    document_embedding_cache=store, # 로컬 저장소 지정
    namespace=embedding.model, # 임베딩 값 구분자 (생략 가능)
)


/home/sunsnu/miniconda3/envs/openai/lib/python3.12/site-packages/langchain/embeddings/cache.py:58: UserWarning: Using default key encoder: SHA‑1 is *not* collision‑resistant. While acceptable for most cache scenarios, a motivated attacker can craft two different payloads that map to the same cache key. If that risk matters in your environment, supply a stronger encoder (e.g. SHA‑256 or BLAKE2) via the `key_encoder` argument. If you change the key encoder, consider also creating a new cache, to avoid (the potential for) collisions with existing keys.
  _warn_about_sha1_encoder()


In [16]:
from langchain_community.document_loaders import UnstructuredFileLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter


loader = UnstructuredFileLoader("./ADsP - 정리.docx")
documents = loader.load()
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)

documents_spliter = text_splitter.split_documents(documents) #



In [17]:
%time db = FAISS.from_documents(documents_spliter, cached_embedder)
%time db2 = FAISS.from_documents(documents_spliter, cached_embedder) #

CPU times: user 72.7 ms, sys: 25.7 ms, total: 98.4 ms
Wall time: 1.67 s
CPU times: user 7.64 ms, sys: 3.88 ms, total: 11.5 ms
Wall time: 11.5 ms


In [21]:
from langchain_huggingface.embeddings import HuggingFaceEmbeddings



model_name_local = "intfloat/multilingual-e5-large-instruct" #
hf_embeddings_local = HuggingFaceEmbeddings(
    model_name=model_name_local,
    model_kwargs={"device": "cuda"}, # GPU가 없다면 "cpu", 맥북은 "mps", 엔비디아 GPU는 "cuda"
    encode_kwargs={"normalize_embeddings": True}, #
)


In [22]:
import os
os.environ["HF_HOME"] = "./cache2/"


texts = [ 
    "안녕, 만나서 반가워.",
    "LangChain simplifies the process of building applications with large language models",
    "랭체인 한국어 튜토리얼은 LangChain의 공식 문서, cookbook 및 다양한 실용 예제를 바탕으로 하여 사용자가 LangChain을 더 쉽고 효과적으로 활용할 수 있도록 구성되어 있습니다.",
    "LangChain은 초거대 언어모델로 애플리케이션을 구축하는 과정을 단순화합니다.",
    "Retrieval-Augmented Generation (RAG) is an effective technique for improving AI responses.",
]


In [23]:
print("\n--- HuggingFaceEmbeddings 문서 임베딩 (로컬 다운로드) ---")
%time embedded_documents_local = hf_embeddings_local.embed_documents(texts) #
print(f"Model: {model_name_local}") #
print(f"Dimension: {len(embedded_documents_local)}") # (출력: 1024)



--- HuggingFaceEmbeddings 문서 임베딩 (로컬 다운로드) ---
CPU times: user 422 ms, sys: 79 ms, total: 501 ms
Wall time: 584 ms
Model: intfloat/multilingual-e5-large-instruct
Dimension: 5


In [25]:
texts = [ #
    "안녕, 만나서 반가워.",
    "LangChain simplifies the process of building applications with large language models",
    "랭체인 한국어 튜토리얼은 LangChain의 공식 문서, cookbook 및 다양한 실용 예제를 바탕으로 하여 사용자가 LangChain을 더 쉽고 효과적으로 활용할 수 있도록 구성되어 있습니다.",
    "LangChain은 초거대 언어모델로 애플리케이션을 구축하는 과정을 단순화합니다.",
    "Retrieval-Augmented Generation (RAG) is an effective technique for improving AI responses.",
]


model_name_bge = "BAAI/bge-m3" #
model_kwargs_bge = {"device": "cuda"} #
encode_kwargs_bge = {"normalize_embeddings": True} #


hf_embeddings_bge = HuggingFaceEmbeddings(
    model_name=model_name_bge,
    model_kwargs=model_kwargs_bge,
    encode_kwargs=encode_kwargs_bge
) #


print("\n--- BGE-M3 임베딩 ---")
%time embedded_documents_bge = hf_embeddings_bge.embed_documents(texts) #
print(f"Model: {model_name_bge}") #
print(f"Dimension: {len(embedded_documents_bge)}") # (출력: 1024)



--- BGE-M3 임베딩 ---
CPU times: user 38.9 ms, sys: 5.23 ms, total: 44.1 ms
Wall time: 43.4 ms
Model: BAAI/bge-m3
Dimension: 5


ollama pull nomic-embed-text


In [28]:
# ollama pull nomic-embed-text

from langchain_community.embeddings import OllamaEmbeddings
ollama_embeddings = OllamaEmbeddings(model="nomic-embed-text") #

query_text_ollama = "LangChain에 대해서 상세히 알려 주세요."
embedded_query_ollama = ollama_embeddings.embed_query(query_text_ollama) #
print(f"Ollama 쿼리 임베딩 차원: {len(embedded_query_ollama)}") # (출력: 768)

Ollama 쿼리 임베딩 차원: 768


pytube를 이용한 유튜브 영상 활용

In [ ]:
from pytubefix import YouTube
from openai import OpenAI

url = "https://www.youtube.com/watch?v=O-ZfXbfAMKU"
